In [1]:
! git clone https://github.com/lim-eren/CDnet2014.git 

Cloning into 'CDnet2014'...
remote: Counting objects: 81, done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 81
Unpacking objects: 100% (81/81), done.


In [4]:
! ls */train/*

CDnet2014/train/x:
in000685.jpg  in000853.jpg  in001057.jpg  in001217.jpg	in001537.jpg
in000765.jpg  in000873.jpg  in001077.jpg  in001364.jpg	in001613.jpg
in000789.jpg  in000896.jpg  in001104.jpg  in001416.jpg	in001636.jpg
in000809.jpg  in000925.jpg  in001125.jpg  in001477.jpg	in001657.jpg
in000832.jpg  in001009.jpg  in001189.jpg  in001517.jpg	in001680.jpg

CDnet2014/train/y:
gt000685.png  gt000853.png  gt001057.png  gt001217.png	gt001537.png
gt000765.png  gt000873.png  gt001077.png  gt001364.png	gt001613.png
gt000789.png  gt000896.png  gt001104.png  gt001416.png	gt001636.png
gt000809.png  gt000925.png  gt001125.png  gt001477.png	gt001657.png
gt000832.png  gt001009.png  gt001189.png  gt001517.png	gt001680.png


In [9]:
# load data func

from __future__ import print_function
import glob, os, numpy as np
from keras.preprocessing import image as kImage

def getData(dataset_dir):
    X_list= sorted(glob.glob(os.path.join(dataset_dir, 'x','*.jpg')))
    Y_list = sorted(glob.glob(os.path.join(dataset_dir, 'y' ,'*.png')))
    
    X= []
    Y= []
    for i in range(len(X_list)):
        # Load input image
        x = kImage.load_img(X_list[i])
        x = kImage.img_to_array(x)
        X.append(x)
        
        # Load ground-truth label and encode to value 0 and 1
        x = kImage.load_img(Y_list[i], grayscale = True)
        x = kImage.img_to_array(x)
        x /= 255.0
        x = np.floor(x)
        Y.append(x)
        
    X = np.asarray(X)
    Y = np.asarray(Y)
    
    # Shuffle the training data
    idx = list(range(X.shape[0]))
    np.random.shuffle(idx)
    X = X[idx]
    Y = Y[idx]
    
    return X, Y

Using TensorFlow backend.


In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# replace YOUR_FILE_ID with your file's id
download = drive.CreateFile({'id': 'YOUR_FILE_ID'})
download.GetContentFile('CDnet2014.zip')

# unzip your file
! unzip CDnet2014.zip

In [15]:
! ls

CDnet2014  CDnet2014.zip  datalab  __MACOSX


In [0]:
import keras
from keras.models import Model
from keras.layers import Deconv2D, Input
import keras.backend as K
import tensorflow as tf
from keras import regularizers

def initModel():
    ### Encoder
    net_input = Input(shape=(240,320,3))
                
    vgg16 = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_tensor=net_input)
    for layer in vgg16.layers[:17]:
      layer.trainable = False
    
    x = vgg16.layers[-2].output # 2nd layer from the last, block5_conv3
    
    ### Decoder
    x = Deconv2D(256, (3,3), strides=(2,2), activation='relu', padding='same')(x)
    x = Deconv2D(128, (3,3), strides=(2,2), activation='relu', padding='same')(x)
    x = Deconv2D(64, (3,3), strides=(2,2), activation='relu', padding='same')(x)
    x = Deconv2D(32, (3,3), strides=(2,2), activation='relu', padding='same')(x)
    x = Deconv2D(1, (1,1), activation='sigmoid', padding='same')(x)
    
    model = Model(inputs=vgg16.input, outputs=x)
    model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.RMSprop(lr=5e-4), metrics=['accuracy'])
    return model

In [5]:
# load data
dataset_path = os.path.join('CDnet2014', 'train')
X, Y = getData(dataset_path)

# init the model
model = initModel()

early = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10)
reduce = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)

model.fit(X, Y, batch_size=1, epochs=100, verbose=2, validation_split=0.2, callbacks=[reduce, early], shuffle=True)
model.save('my_model.h5')

Train on 20 samples, validate on 5 samples
Epoch 1/100
 - 3s - loss: 0.2404 - acc: 0.8853 - val_loss: 0.1378 - val_acc: 0.9362
Epoch 2/100
 - 1s - loss: 0.0969 - acc: 0.9616 - val_loss: 0.0622 - val_acc: 0.9748
Epoch 3/100
 - 1s - loss: 0.0648 - acc: 0.9731 - val_loss: 0.0678 - val_acc: 0.9684
Epoch 4/100
 - 1s - loss: 0.0506 - acc: 0.9775 - val_loss: 0.0519 - val_acc: 0.9784
Epoch 5/100
 - 1s - loss: 0.0509 - acc: 0.9786 - val_loss: 0.0693 - val_acc: 0.9707
Epoch 6/100
 - 1s - loss: 0.0398 - acc: 0.9824 - val_loss: 0.0565 - val_acc: 0.9788
Epoch 7/100
 - 1s - loss: 0.0454 - acc: 0.9808 - val_loss: 0.0456 - val_acc: 0.9827
Epoch 8/100
 - 1s - loss: 0.0325 - acc: 0.9863 - val_loss: 0.0559 - val_acc: 0.9804
Epoch 9/100
 - 1s - loss: 0.0324 - acc: 0.9871 - val_loss: 0.0636 - val_acc: 0.9793
Epoch 10/100
 - 1s - loss: 0.0288 - acc: 0.9877 - val_loss: 0.0671 - val_acc: 0.9783
Epoch 11/100
 - 1s - loss: 0.0267 - acc: 0.9887 - val_loss: 0.0454 - val_acc: 0.9826
Epoch 12/100
 - 1s - loss: 0.02

In [75]:
from keras import backend as K
import tensorflow as tf

dataset_path = os.path.join('CDnet2014', 'test')
X, Y = getData(dataset_path)

pred = model.predict(X, verbose=1, batch_size=1)
print(tf.Session().run(K.mean(K.equal(Y, K.round(pred)))))

10/10 [==============================] - 1s 56ms/step
0.9896263


In [0]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12.0, 9.0)

idx = 1 # image index in a range of [0,9]

img = np.empty(3, dtype=object)
img[0] = X[idx]
img[1] = Y[idx].reshape(Y[idx].shape[0],Y[idx].shape[1])
img[2] = pred[idx].reshape(pred[idx].shape[0],pred[idx].shape[1])
title = ['input', 'ground-truth', 'result']
for i in range(3):
  plt.subplot(1, 3, i+1)
  if i==0:
    plt.imshow(img[i].astype('uint8'))
  else:
    plt.imshow(img[i], cmap='gray')
  
  plt.axis('off')
  plt.title(title[i])
plt.show()